# ARIMA training and prediction on encrypted data
Expected RAM usage: 20 GB.
Expected runtime: 30-60 seconds.

## Introduction

This example demonstrates FHE training of an ARIMA(1,1,1) model on encrypted time series values. The train model is also used to predict the next value of the time series. The FHE prediction is compared with the plain ARIMA(1,1,1) prediction and the prediction results are shown to be close.

Verify that HEaaN backend is available

In [ ]:
import sys
import pyhelayers
if not hasattr(pyhelayers, "HeaanContext"):
    print("This demo requires HEaaN backend which is not yet available for this platform")
    sys.exit(0)

## Step 1. Client side - initialize ARIMA model

### 1.1. Generate time series values of an ARIMA(1,1,1) model, in the plain.

In [ ]:
import math
import numpy as np
import os
import utils

mu = 50
phi1 = 0.6
phi2 = -0.4
M = mu/(1-phi1)
N = 2 ** 15

errors = np.random.normal(0, 1, N)
X_train = [M]
for t in range(1,N):
    X_train.append(mu + phi1*X_train[t-1] + phi2*errors[t-1] + errors[t])
X_train = np.array(X_train).reshape([len(X_train), 1])

### 1.2 Load the hyperparameters of the ARIMA(1,1,1) model
These hyperparameters configure the FHE training and prediction algorithms.

In [ ]:
hyper_params_file = os.path.join(utils.get_data_sets_dir(), 'arima', 'model.json')
hyper_params = pyhelayers.PlainModelHyperParams()
hyper_params.load(hyper_params_file)

## 1.3 Define HE run requirements
These requirements specify how the HE encryption should be configured. Here, we require the HE encryption to be done with HEaaN encryption scheme.

In [ ]:
he_run_req = pyhelayers.HeRunRequirements()
he_run_req.set_he_context_options([pyhelayers.HeaanContext()])

### 1.4 Initialize an HE Arima model.
We initialized the HE Arima model with the above created HE run requirements. Calling `encode_encrypt` below activates an internal optimization process which finds the best HE-related configuration satisfying the given requirement.

In [ ]:
he_arima = pyhelayers.Arima()
he_arima.encode_encrypt(files=[], he_run_req=he_run_req, hyper_params=hyper_params)
he_context = he_arima.get_created_he_context()

### 1.5 Get a `ModelIoEncoder` from the HE model.
The ModelIoEncoder objects will be used to encrypt and decrypt the input and output of the training and prediction.

In [ ]:
model_io_encoder = pyhelayers.ModelIoEncoder(he_arima)

### 1.6 Encrypt the training input

In [ ]:
X_train_enc = pyhelayers.EncryptedData(he_context)
model_io_encoder.encode_encrypt(X_train_enc, [X_train])

### 1.7 Save the initialized HE Arima model and encrypted input to a buffers

In [ ]:
he_arima_buf = he_arima.save_to_buffer()
X_train_enc_buf = X_train_enc.save_to_buffer()

## Step 2. Server side - train the He Arima model over encrypted data

### 2.1 Load the initialized HE Arima model and encrypted input

In [ ]:
server_he_arima = pyhelayers.load_he_model(he_context, he_arima_buf)
server_x_train_enc = pyhelayers.load_encrypted_data(he_context, X_train_enc_buf)

### 2.2 Train the Arima model over encrypted data
This step results with a trained Arima model whose weights are encrypted.

In [ ]:
server_he_arima.fit(server_x_train_enc)

### 2.3 Save the trained Arima model

In [ ]:
fit_arima_buf = server_he_arima.save_to_buffer()

## Step 3. Client side - decrypt training results and encrypt prediction input.
We will decrypt the above trained model and encrypt the time series we want to predict its next value. We demonstrate a use case in which the prediction input is owned by a separate entity. This separate entity will encrypt its input and send it to the server side for prediction.

## 3.1 Load and decrypt the trained Arima model

In [ ]:
he_arima = pyhelayers.load_he_model(he_context, fit_arima_buf)
plain_arima = he_arima.decrypt_decode()

### 3.2 Build a new HE run requirements.
The HE run requirements specify how the HE encryption should be configured. We build new requirements that are tailored for prediction. This time, we choose to not encrypt the model weights.

In [ ]:
he_run_req2 = pyhelayers.HeRunRequirements()
he_run_req2.set_he_context_options([pyhelayers.HeaanContext()])
he_run_req2.set_model_encrypted(False)

## 3.3 Compile the plain model and HE run requirements into HE profile
This HE profile holds encryption-specific parameters.

In [ ]:
profile = pyhelayers.HeModel.compile(plain_arima, he_run_req2)

### 3.4 Initialize an HE Arima model.
We initialized the HE Arima model using the plain Arima model and the HE profile computed above.

In [ ]:
he_context = pyhelayers.HeModel.create_context(profile)
he_arima = plain_arima.get_empty_he_model(he_context)

### 3.5 Encode the HE Arima model
We encode the HE Arima model using the weights from the plain Arima model.


In [ ]:
he_arima.encode(plain_arima, profile)

## 3.6 Create and save an `ModelIoEncoder` object
This ModelIoEncoder can be sent to a separate entity that owns the prediction input. The separate entity then uses this IoProcessor to encrypt its data and send it to the server for prediction.

In [ ]:
model_io_encoder = pyhelayers.ModelIoEncoder(he_arima)
model_io_encoder_buf = model_io_encoder.save_to_buffer()

### 3.7 Encrypt the prediction input
We load the ModelIoEncoder object saved above and use it to encrypt the prediction input.

In [ ]:
predict_input = X_train[-hyper_params.num_values_used_for_prediction:]
model_io_encoder = pyhelayers.load_io_encoder(he_context, model_io_encoder_buf)
predict_input_enc = pyhelayers.EncryptedData(he_context)
model_io_encoder.encode_encrypt(predict_input_enc, [predict_input])

## 3.8 Save the encoded Arima model and input

In [ ]:
he_arima_buf = he_arima.save_to_buffer()
predict_input_enc_buf = predict_input_enc.save_to_buffer()

## Step 4. Server side - predict over encrypted data

### 4.1 Load the encoded HE Arima model and input
This model includes plaintext weights, and it will be used to run prediction over encrypted data.

In [ ]:
he_arima = pyhelayers.load_he_model(he_context, he_arima_buf)
predict_input_enc = pyhelayers.load_encrypted_data(he_context, predict_input_enc_buf)

## 4.2 Run FHE prediction.
This step returns an encrypted prediction result.

In [ ]:
print('Homomorphically predicting the next value in the time series . . .')
res_enc = pyhelayers.EncryptedData(he_context)
he_arima.predict(res_enc, predict_input_enc)
res_enc_buf = res_enc.save_to_buffer()

## Step 5. Client side - decrypt and assess result

## 5.1 Decrypt the prediction result

In [ ]:
print('Decrypting the prediction result . . .')
res_enc = pyhelayers.load_encrypted_data(he_context, res_enc_buf)
res = model_io_encoder.decrypt_decode_output(res_enc)
fhe_prediction = res[0]

## 5.2 Compute the expected prediction, in the plain

In [ ]:
plain_prediction = mu + phi1 * X_train[-1] + phi2 * errors[-1]

## 5.2 Compare the FHE prediction with the expected plain prediction


In [ ]:
print('FHE ARIMA(1,1,1) prediction = ', fhe_prediction)
print('plain ARIMA(1,1,1) prediction = ', plain_prediction)
absolute_diff = math.fabs(fhe_prediction - plain_prediction)
relative_diff = absolute_diff / math.fabs(plain_prediction)
print('absolute diff  = ', absolute_diff)
print('relative_diff = ', relative_diff)

In [ ]:
print("RAM usage:", utils.get_used_ram(), "MB")